# Convert from json to table

In [1]:
import json
import os
import pandas as pd

In [22]:
data_folder = 'ift6758/data/raw/NHL/'
# for filename in os.listdir(data_folder):
f = open(data_folder+'20162017.json')
data = json.load(f)
i = 0
print(type(data))

# for line in data:
#     i+=1
#     print(line)
#     if i>3: break



<class 'dict'>


In [24]:
data[list(data.keys())[0]][0].keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [56]:
data['20162017'][0]['liveData']['plays']['allPlays']



{'players': [{'player': {'id': 8477939,
    'fullName': 'William Nylander',
    'link': '/api/v1/people/8477939'},
   'playerType': 'Blocker'},
  {'player': {'id': 8470602,
    'fullName': 'Dion Phaneuf',
    'link': '/api/v1/people/8470602'},
   'playerType': 'Shooter'}],
 'result': {'event': 'Blocked Shot',
  'eventCode': 'OTT204',
  'eventTypeId': 'BLOCKED_SHOT',
  'description': 'William Nylander blocked shot from Dion Phaneuf'},
 'about': {'eventIdx': 10,
  'eventId': 204,
  'period': 1,
  'periodType': 'REGULAR',
  'ordinalNum': '1st',
  'periodTime': '02:05',
  'periodTimeRemaining': '17:55',
  'dateTime': '2016-10-12T23:20:54Z',
  'goals': {'away': 0, 'home': 0}},
 'coordinates': {'x': 56.0, 'y': -2.0},
 'team': {'id': 10,
  'name': 'Toronto Maple Leafs',
  'link': '/api/v1/teams/10',
  'triCode': 'TOR'}}

In [78]:

import pandas as pd
import json


class NHLConverter:
    """
    Class to convert data from raw json to interpretable table with following constraints
    "all events of every game into a pandas dataframe,include events of the type “shots” and “goals”, ignore missed shots or blocked shots for now"
    Features :  
        - game time/period information, 
        - game ID, team information (which team took the shot), 
        - indicator if its a shot or a goal, 
        - the on-ice coordinates, 
        - the shooter  
        - goalie name (don’t worry about assists for now), 
        - shot type, 
        - if it was on an empty net
        - whether or not a goal was at even strength, shorthanded, or on the power play
    """
    # liveData/plays/
    def __init__(self, json_fn: str) -> None:
        self.json_fn = json_fn
        
    def pull_json(self) -> None:  
        f = open(self.json_fn)
        loaded_data = json.load(f)

        if len(loaded_data.keys()) == 1:        # Store dictionary with shot info, remove copyright
            # print(loaded_data[list(loaded_data.keys())[0]])
            
            self.raw_data = loaded_data[list(loaded_data.keys())[0]]
            print(len(self.raw_data))
        else:
            print('Wrong format')


    def convert_season(self) -> pd.DataFrame:

        cols = ['game starttime', 'game endtime', 'gameId','offense_team_id', 'offense_team_name', 'offense_team_tricode','goal', 'x_coords', 'y_coords', 'goalie_id', 'goalie_name', 'shooter_id', 'shooter_name', 'shot type', 'empty net', 'strength_shorthand', 'strength_even', 'strength_powerplay']
        selec_data = []
        game_id = 0
        for game in self.raw_data:
            # print(game_id)
            game_endtime = game['gameData']['datetime']['endDateTime']
            game_starttime = game['gameData']['datetime']['dateTime']
            
            event_list = game['liveData']['plays']['allPlays']
            
            for event in event_list: 
                if event['result']['event'] == 'Shot' or event['result']['event'] == 'Goal':

                    curr_event = [game_starttime, game_endtime, game_id]
                    curr_event.append(event['team']['id'])
                    curr_event.append(event['team']['name'])
                    curr_event.append(event['team']['triCode'])
                    # curr_event.append(event['result']['event'])
                    curr_event.append(0 if event['result']['event']== 'Shot' else 1)
                    curr_event.append(event['coordinates'].get('x', None))
                    curr_event.append(event['coordinates'].get('y', None))
                    for p in event['players']:
                        if p['playerType'] == 'Goalie':
                            goalie_id = p['player']['id']
                            goalie_name = p['player']['fullName']
                        elif p['playerType'] == 'Shooter' or p['playerType'] == 'Scorer':
                            shooter_id = p['player']['id']
                            shooter_name = p['player']['fullName']
                    curr_event.append(goalie_id)
                    curr_event.append(goalie_name)
                    curr_event.append(shooter_id)
                    curr_event.append(shooter_name)
                    
                    curr_event.append(event['result'].get('secondaryType', None))

                    if event['result']['event'] == 'Goal':
                        curr_event.append(event['result'].get('emptyNet', None))
                        ## 
                        if event['result']['strength'].get('code', None) == 'EVEN': 
                            curr_event.append(0)
                            curr_event.append(1)
                            curr_event.append(0)
                        elif event['result']['strength'].get('code', None) == 'PPG': 
                            curr_event.append(0)
                            curr_event.append(0)
                            curr_event.append(1)
                        else: 
                            curr_event.append(1)
                            curr_event.append(0)
                            curr_event.append(0)
                    else:
                        curr_event.append(False)
                        curr_event.append(None)

                    
                    selec_data.append(curr_event)
                    game_id+=1
        print(game_id)

        return pd.DataFrame(selec_data, columns=cols)

        
    

In [79]:
# pd.DataFrame.from_dict(data['20162017'][0]['liveData']['plays']['allPlays'][0]['result'], orient='index')
# pd.DataFrame.from_dict(data['20162017'][0]['liveData']['plays']['allPlays'][0]['about'], orient='index')
# game = data['20162017'][0]

# from ift6758.data import NH?LConverter
converter = NHLConverter('ift6758/data/raw/NHL/20162017.json')

converter.pull_json()
# convert_season()

# json.loads('/ift6?758/data/raw/NHL/')


1317


In [80]:
clean = converter.convert_season()

80399


In [83]:
clean.iloc[:10, :]

,game starttime,game endtime,gameId,offense_team_id,offense_team_name,offense_team_tricode,goal,x_coords,y_coords,goalie_id,goalie_name,shooter_id,shooter_name,shot type,empty net,strength
0,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,0,10,Toronto Maple Leafs,TOR,0,-77.0,5.0,8467950,Craig Anderson,8478483,Mitchell Marner,Wrist Shot,False,NaN
1,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,1,9,Ottawa Senators,OTT,0,86.0,13.0,8475883,Frederik Andersen,8467967,Chris Kelly,Wrist Shot,False,NaN
2,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,2,9,Ottawa Senators,OTT,0,23.0,-38.0,8475883,Frederik Andersen,8476879,Cody Ceci,Wrist Shot,False,NaN
3,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,3,9,Ottawa Senators,OTT,0,33.0,-15.0,8475883,Frederik Andersen,8474578,Erik Karlsson,Slap Shot,False,NaN
4,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,4,10,Toronto Maple Leafs,TOR,0,-34.0,28.0,8467950,Craig Anderson,8475716,Martin Marincin,Wrist Shot,False,NaN
5,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,5,10,Toronto Maple Leafs,TOR,0,-33.0,-17.0,8467950,Craig Anderson,8478483,Mitchell Marner,Wrist Shot,False,NaN
6,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,6,10,Toronto Maple Leafs,TOR,1,-70.0,1.0,8467950,Craig Anderson,8479318,Auston Matthews,Wrist Shot,False,1.0
7,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,7,10,Toronto Maple Leafs,TOR,0,-45.0,-36.0,8467950,Craig Anderson,8474709,Matt Martin,Wrist Shot,False,NaN
8,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,8,9,Ottawa Senators,OTT,0,33.0,-18.0,8475883,Frederik Andersen,8474578,Erik Karlsson,Slap Shot,False,NaN
9,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,9,9,Ottawa Senators,OTT,0,34.0,20.0,8475883,Frederik Andersen,8474578,Erik Karlsson,Wrist Shot,False,NaN
